Code to Analyse width of Cracks

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
from pathlib import Path
from skimage.morphology import skeletonize
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm

# ===============================================================
# ================= 1. CORE FUNCTIONS ===========================
# ===============================================================

def crack_width_measure(binary_image, pixel_size_mm=0.1):
    """Measures the maximum crack width from a binary image."""
    if np.sum(binary_image) == 0: return 0
    skeleton = skeletonize(binary_image > 0)
    distance = distance_transform_edt(binary_image)
    if not np.any(skeleton): return 0
    crack_widths_pixels = distance[skeleton] * 2
    crack_widths_mm = crack_widths_pixels * pixel_size_mm
    return np.max(crack_widths_mm)

def get_width_from_image(image_path, pixel_size_mm=0.1):
    """Reads an image and returns its max crack width."""
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if img is None: return None
    _, binary_image = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    max_width = crack_width_measure(binary_image, pixel_size_mm)
    return max_width

def get_widths_from_folder(folder_path, pixel_size_mm=0.1):
    """Processes a folder of images and returns a dictionary of their widths."""
    results = {}
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for filename in tqdm(image_files, desc=f"Analyzing {Path(folder_path).name}"):
        image_path = os.path.join(folder_path, filename)
        max_width = get_width_from_image(image_path, pixel_size_mm)
        if max_width is not None:
            results[filename] = max_width
    return results

# ===============================================================
# =========== 2. DEFINE FOLDERS AND RUN ANALYSIS ================
# ===============================================================

# --- Set Your Folders and Calibration Factor Here ---
folder_clean = Path("Dataset_1000/")
folder_shadowed = Path("shadow/")
folder_restored = Path("unshadowed_results/")
PIXEL_TO_MM_RATIO = 0.01 # Adjust if needed

# --- Run Analysis on Each Folder ---
results_clean = get_widths_from_folder(str(folder_clean), pixel_size_mm=PIXEL_TO_MM_RATIO)
results_shadowed = get_widths_from_folder(str(folder_shadowed), pixel_size_mm=PIXEL_TO_MM_RATIO)
results_restored = get_widths_from_folder(str(folder_restored), pixel_size_mm=PIXEL_TO_MM_RATIO)

# --- Combine and Compare Results ---
comparison_data = []

for clean_filename, clean_width in results_clean.items():
    base_name = Path(clean_filename).stem
    
    # Construct expected filenames
    shadow_filename = f"shadow_{clean_filename}"
    restored_filename_jpg = f"shadow_{clean_filename}"
    restored_filename_png = f"shadow_{base_name}.png" # Assuming GAN saves as PNG
    
    # Get the corresponding widths from the other dictionaries
    shadow_width = results_shadowed.get(shadow_filename)
    restored_width = results_restored.get(restored_filename_jpg, results_restored.get(restored_filename_png))

    comparison_data.append({
        'filename': clean_filename,
        'original_width_mm': clean_width,
        'shadowed_width_mm': shadow_width,
        'restored_width_mm': restored_width
    })

# Create and save the final DataFrame
df_comparison = pd.DataFrame(comparison_data).round(4) # Round to 4 decimals

print("\n--- Shadow Width Comparison Report ---")
print(df_comparison.head())

output_csv_path = "shadow.csv"
df_comparison.to_csv(output_csv_path, index=False)
print(f"\n✅ Final comparison report saved to '{output_csv_path}'")

Analyzing unshadowed_results:  61%|██████▏   | 613/1000 [00:05<00:03, 112.06it/s]

Blur

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
from pathlib import Path
from skimage.morphology import skeletonize
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm

# ===============================================================
# ================= 1. CORE FUNCTIONS ===========================
# ===============================================================

def crack_width_measure(binary_image, pixel_size_mm=0.1):
    """Measures the maximum crack width from a binary image."""
    if np.sum(binary_image) == 0: return 0
    skeleton = skeletonize(binary_image > 0)
    distance = distance_transform_edt(binary_image)
    if not np.any(skeleton): return 0
    crack_widths_pixels = distance[skeleton] * 2
    crack_widths_mm = crack_widths_pixels * pixel_size_mm
    return np.max(crack_widths_mm)

def get_width_from_image(image_path, pixel_size_mm=0.1):
    """Reads an image and returns its max crack width."""
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if img is None: return None
    _, binary_image = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    max_width = crack_width_measure(binary_image, pixel_size_mm)
    return max_width

def get_widths_from_folder(folder_path, pixel_size_mm=0.1):
    """Processes a folder of images and returns a dictionary of their widths."""
    results = {}
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for filename in tqdm(image_files, desc=f"Analyzing {Path(folder_path).name}"):
        image_path = os.path.join(folder_path, filename)
        max_width = get_width_from_image(image_path, pixel_size_mm)
        if max_width is not None:
            results[filename] = max_width
    return results

# ===============================================================
# =========== 2. DEFINE FOLDERS AND RUN ANALYSIS ================
# ===============================================================

# --- Set Your Folders and Calibration Factor Here ---
folder_clean = Path("Dataset_1000/")
folder_shadowed = Path("blur/")
folder_restored = Path("unshadowed_results/")
PIXEL_TO_MM_RATIO = 0.01 # Adjust if needed

# --- Run Analysis on Each Folder ---
results_clean = get_widths_from_folder(str(folder_clean), pixel_size_mm=PIXEL_TO_MM_RATIO)
results_shadowed = get_widths_from_folder(str(folder_shadowed), pixel_size_mm=PIXEL_TO_MM_RATIO)
results_restored = get_widths_from_folder(str(folder_restored), pixel_size_mm=PIXEL_TO_MM_RATIO)

# --- Combine and Compare Results ---
comparison_data = []

for clean_filename, clean_width in results_clean.items():
    base_name = Path(clean_filename).stem
    
    # Construct expected filenames
    shadow_filename = f"shadow_{clean_filename}"
    restored_filename_jpg = f"shadow_{clean_filename}"
    restored_filename_png = f"shadow_{base_name}.png" # Assuming GAN saves as PNG
    
    # Get the corresponding widths from the other dictionaries
    shadow_width = results_shadowed.get(shadow_filename)
    restored_width = results_restored.get(restored_filename_jpg, results_restored.get(restored_filename_png))

    comparison_data.append({
        'filename': clean_filename,
        'original_width_mm': clean_width,
        'shadowed_width_mm': shadow_width,
        'restored_width_mm': restored_width
    })

# Create and save the final DataFrame
df_comparison = pd.DataFrame(comparison_data).round(4) # Round to 4 decimals

print("\n--- Shadow Width Comparison Report ---")
print(df_comparison.head())

output_csv_path = "shadow_width_comparison_simple.csv"
df_comparison.to_csv(output_csv_path, index=False)
print(f"\n✅ Final comparison report saved to '{output_csv_path}'")